In [1]:
import numpy as np
from scipy.special import jv, yv
import pandas as pd

# LSRK5 係數 (Carpenter & Kennedy 1994)
A = np.array([0.0,
              -567301805773.0 / 1357537059087.0,
              -2404267990393.0 / 2016746695238.0,
              -3550918686646.0 / 2091501179385.0,
              -1275806237668.0 / 842570457699.0])

B = np.array([1432997174477.0 / 9575080441755.0,
              5161836677717.0 / 13612068292357.0,
              1720146321549.0 / 2090206949498.0,
              3134564353537.0 / 4481467310338.0,
              2277821191437.0 / 14882151754819.0])

C = np.array([0.0, 0.1496590219993, 0.3704009573644, 0.6222557631345, 0.9582821306748])

# 問題參數
n = 8
c = 1.0
alpha = 6.1556542986056666
omega = alpha * c

def exact_solution(r, theta, t, n, alpha, omega, hat_u):
    """
    解析解 - 矩陣維度 (K, N+1)
    Args:
        r, theta: meshgrid 座標矩陣，形狀 (K, N+1)
        t: 時間
        n, alpha, omega: 問題參數
        hat_u: 徑向函數值，形狀 (K,)
    Returns:
        u, u_t: 解和時間導數，形狀 (K, N+1)
    """
    hat_u_expanded = hat_u[:, np.newaxis]  # (K, 1) 廣播為 (K, N+1)
    u = hat_u_expanded * np.cos(n * theta - omega * t)
    u_t = omega * hat_u_expanded * np.sin(n * theta - omega * t)
    return u, u_t

def f_r_analytic(r, theta, t, n, alpha, omega, hat_u):
    """
    徑向項解析形式 - 按照 main_organized.md Part2 公式
    f_r(u(r,θ,t)) = (1/r)(∂/∂r)(r(∂û(r)/∂r))cos(8θ-ωt)
    
    根據分離變數法，對於解析解 û(r) = A*J₈(αr) + B*Y₈(αr):
    (1/r)(∂/∂r)(r(∂û/∂r)) = (64/r² - α²)û(r)
    """
    hat_u_expanded = hat_u[:, np.newaxis]
    # 正確的公式：(64/r² - α²)û
    radial_term = (64/r**2 - alpha**2) * hat_u_expanded
    return radial_term * np.cos(n * theta - omega * t)

def f_theta(u, r, theta, t, n, alpha, omega, hat_u):
    """
    角度項解析形式 - 按照 main_organized.md Part2 公式
    f_θ(u(r,θ,t)) = -(64/r²)û(r)cos(8θ-ωt)
    """
    hat_u_expanded = hat_u[:, np.newaxis]
    return -(64/r**2) * hat_u_expanded * np.cos(n * theta - omega * t)

def rhs_system(u, v, r, theta, t, n, alpha, omega, c, r_arr, hat_u):
    """
    系統右端項
    Args:
        u, v: 解及其時間導數，形狀 (K, N+1)
        r, theta: 座標矩陣，形狀 (K, N+1)
        hat_u: 徑向函數值，形狀 (K,)
        其他參數...
    Returns:
        du_dt, dv_dt: 時間導數，形狀 (K, N+1)
    """
    du_dt = v
    dv_dt = f_r_analytic(r, theta, t, n, alpha, omega, hat_u) + f_theta(u, r, theta, t, n, alpha, omega, hat_u)
    return du_dt, dv_dt
    
# Part II: 低儲存量 RK 方法的時間推進 - 按照 main_organized.md Part2 要求
K = 20
T = 2 * np.pi / alpha

# Part2 要求：固定格點解 (K, N+1) = (20, 64)
N_plus_1 = 64

# Part2 要求：CFL = 1, 0.5, 0.25, 0.125
CFL_list = [1.0, 0.5, 0.25, 0.125]
errors = []

A_coef = 1.0
B_coef = -A_coef * jv(n, alpha * 1) / yv(n, alpha * 1)

# 網格設置 - 使用 K 個內部點
r = np.array([1 + (k+1) / (K+1) for k in range(K)])  # K 個內部點
hat_u = A_coef * jv(n, alpha * r) + B_coef * yv(n, alpha * r)
theta = np.array([2 * np.pi * n / N_plus_1 for n in range(N_plus_1)])  # θ_n = 2πn/(N+1) (N+1個點)
dr = 1 / K
dtheta = 2 * np.pi / N_plus_1

# 矩陣排列：行對應徑向r，列對應角度θ - 產生 (K, N+1) 維度的矩陣
R, Theta = np.meshgrid(r, theta, indexing='ij')  # R.shape = (K, N+1), Theta.shape = (K, N+1)
r_arr = r  # 徑向坐標陣列

for CFL in CFL_list:
    # 時間步進
    dt = CFL * min(dtheta, dr)
    num_steps = int(np.ceil(T / dt))
    dt = T / num_steps
    
    # 初始條件
    u, v = exact_solution(R, Theta, 0, n, alpha, omega, hat_u)
    
    # LSRK5 時間步進
    for step in range(num_steps):
        t_n = step * dt
        u_stage, v_stage = u.copy(), v.copy()
        du, dv = np.zeros_like(u), np.zeros_like(v)
        
        for j in range(5):
            t_stage = t_n + C[j] * dt
            du_dt, dv_dt = rhs_system(u_stage, v_stage, R, Theta, t_stage, n, alpha, omega, c, r_arr, hat_u)
            du = A[j] * du + dt * du_dt
            dv = A[j] * dv + dt * dv_dt
            u_stage += B[j] * du
            v_stage += B[j] * dv
        
        u, v = u_stage, v_stage
    
    # 誤差計算
    u_exact, _ = exact_solution(R, Theta, T, n, alpha, omega, hat_u)
    max_error = np.max(np.abs(u - u_exact))
    errors.append(max_error)

# 收斂率計算
rates = [np.nan]
for i in range(1, len(errors)):
    # Part2 收斂率：時間步長與 CFL 成比例，所以收斂率基於 CFL 比值
    rate = np.log(errors[i-1] / errors[i]) / np.log(CFL_list[i-1] / CFL_list[i])
    rates.append(rate)

# 結果表格 - 按照 Part2 格式
df = pd.DataFrame({
    "CFL": CFL_list,
    "誤差 (e₀)": errors,
    "收斂率 (C.R.)": rates
})

print("Part II: 低儲存量 RK 方法時間推進收斂性研究")
print("模擬參數: (K, N+1) = (20, 64), T = 2π/α")
print(df.to_string(index=False, float_format="%.6e"))

Part II: 低儲存量 RK 方法時間推進收斂性研究
模擬參數: (K, N+1) = (20, 64), T = 2π/α
         CFL      誤差 (e₀)   收斂率 (C.R.)
1.000000e+00 7.585132e-08          NaN
5.000000e-01 4.297210e-09 4.141702e+00
2.500000e-01 2.685281e-10 4.000255e+00
1.250000e-01 1.678314e-11 3.999988e+00
